In [1]:
import pandas as pd 
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

def developer( desarrollador : str ):   
Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.   
Ejemplo de retorno:  
Año	Cantidad de Items	Contenido Free  
2023	50	                27%  
2022	45	                25%  
xxxx	xx	                xx%  

In [28]:
def developer(desarrollador: str):
    
    """
    Recupera información sobre un desarrollador específico.

    Args:
        desarrollador (str): Nombre del desarrollador.

    Returns:
        pd.DataFrame: DataFrame con información sobre el desarrollador.
    """
    
    if not isinstance(desarrollador, str):
        return print('El argumento "desarrollador" debe ser una cadena de texto (str).')
    
    df_steam = pd.read_parquet('../CleanData/steam_games.parquet')
    df_steam['free'] = df_steam['price'].apply(lambda x: 1 if x == 0 else 0)
    df_steam['Año'] = df_steam['release_date'].dt.year
    
    desarrolladores = df_steam['developer'].str.lower().unique()
    desarrollador = desarrollador.lower()

    if desarrollador not in desarrolladores:
        return print('Desarrollador no encontrado. Inserte un desarrollador válido')

    df = df_steam.groupby(['Año', 'developer']).agg(
                            {'id': 'count', 'free': lambda x: (x.sum() / x.count()) * 100}
                        ).reset_index().rename(columns={'free': 'Contenido Free', 'id': 'Cantidad de Items'})
    
    df['developer'] = df['developer'].apply(lambda x: x.lower())
    
    df = df[df['developer'] == desarrollador].drop(columns= 'developer').sort_values(by= 'Año', ascending=False)
    df['Contenido Free'] = round(df['Contenido Free'], 2)
    df['Contenido Free'] = df['Contenido Free'].apply(lambda x: str(x) + '%')
    df.reset_index(drop=True, inplace=True)

    resultado = {
        f'Año {int(df.loc[i,"Año"])}': {'Cantidad de Items': int(df.loc[i, 'Cantidad de Items']), 'Contenido Free:': df.loc[i, 'Contenido Free']}
        for i in range(len(df))
    }

    return resultado

In [29]:
developer('Digital Leisure Inc.')

{'Año 2015': {'Cantidad de Items': 1, 'Contenido Free:': '100.0%'},
 'Año 2014': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 1991': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 1984': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 1983': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'}}

In [23]:
developer('Valve')

{'Año 2017': {'Cantidad de Items': 2, 'Contenido Free:': '0.0%'},
 'Año 2016': {'Cantidad de Items': 1, 'Contenido Free:': '100.0%'},
 'Año 2014': {'Cantidad de Items': 7, 'Contenido Free:': '0.0%'},
 'Año 2012': {'Cantidad de Items': 2, 'Contenido Free:': '0.0%'},
 'Año 2011': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2010': {'Cantidad de Items': 2, 'Contenido Free:': '50.0%'},
 'Año 2009': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2008': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2007': {'Cantidad de Items': 3, 'Contenido Free:': '33.33%'},
 'Año 2006': {'Cantidad de Items': 2, 'Contenido Free:': '0.0%'},
 'Año 2005': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2004': {'Cantidad de Items': 5, 'Contenido Free:': '0.0%'},
 'Año 2003': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2001': {'Cantidad de Items': 1, 'Contenido Free:': '0.0%'},
 'Año 2000': {'Cantidad de Items': 2, 'Contenido Free:': '0.0%'},
 'Año

def userdata( User_id : str ):  
Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.  
Ejemplo de retorno:   
{"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [216]:
def userdata(user_id: str):

    if not isinstance(user_id, str):
        return print('El argumento user_id debe ser una cadena de texto.')

    df_user = pd.read_parquet('../CleanData/users_items.parquet', columns=['user_id', 'item_id'])
    
    usuarios = df_user['user_id'].unique()

    if user_id not in usuarios:
        return print('Usuario no encontrado. Por favor ingrese un usuario válido')
    
    df_steam = pd.read_parquet('../CleanData/steam_games.parquet', columns= ['id', 'price'])
    df_reviews = pd.read_parquet('../CleanData/reviews.parquet', columns= ['user_id', 'recommend'])

    df_user['Cantidad de Items'] = df_user.groupby('user_id')['user_id'].transform('count')
    
    df_reviews = df_reviews.groupby('user_id').agg('sum').reset_index()

    df = df_user.merge(df_steam, how='left', left_on= 'item_id', right_on= 'id')
    df = df.merge(df_reviews, how = 'left')
    df = df.drop(columns=['item_id', 'id'])
    df = df.groupby('user_id').agg('max').reset_index()
    
    df = df[df['user_id'] == user_id]

    df['recommend'] = df['recommend'].fillna(0)
    df['recommend'] = round(df['recommend'] / df['Cantidad de Items'], 2)
    df['recommend'] = df['recommend'].apply(lambda x: str(x) + ' %')
    df.rename(columns= {'recommend': '% de recomendación'}, inplace=True)
    
    df.rename(columns= {'price': 'Dinero gastado'}, inplace=True)
    df['Dinero gastado'] = df['Dinero gastado'].apply(lambda x: str(x) + ' USD')
    
    df.reset_index(inplace=True, drop=True)

    resultado = {
        'Usuario': df.loc[0, 'user_id'],
        'Dinero gastado': df.loc[0, 'Dinero gastado'],
        '% de recomendación': df.loc[0, '% de recomendación'],
        'Cantidad de Items': int(df.loc[0, 'Cantidad de Items'])
    }
    return resultado

In [217]:
userdata('jonasdbomb')

{'Usuario': 'jonasdbomb',
 'Dinero gastado': '74.76 USD',
 '% de recomendación': '0.0 %',
 'Cantidad de Items': 461}

def UserForGenre( genero : str ):   
Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.  

Ejemplo de retorno:   
{"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,   
"Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [221]:
def UserForGenre(genero: str):
    
    if not isinstance(genero, str):
        return print('El género ingresado debe ser una cadena de texto (string)')

    genero = 'genre_' + genero
    genero = genero.lower()

    df_steam = pd.read_parquet('../CleanData/steam_games.parquet')
    df_steam = df_steam.rename(columns= lambda x: x.lower())

    columnas = list(df_steam.columns)

    if genero not in columnas:
        print('Género no encontrado. Ingrese un género válido.')
        print('Los géneros disponibles son:')
        generos = [col.replace('genre_', '') for col in columnas if 'genre_' in col]
        print(generos)
        return None

    df_user = pd.read_parquet('../CleanData/users_items.parquet', columns=['user_id', 'item_id', 'playtime_forever'])

    df_steam = df_steam[df_steam[genero] == 1][['id','release_date']]
    df_steam['Año'] = df_steam['release_date'].dt.year
    df_steam.drop(columns= 'release_date', inplace= True)

    df = df_user.merge(df_steam, how= 'left', left_on= 'item_id', right_on= 'id')
    df = df.dropna()
    df = df.groupby(['user_id', 'Año']).agg({'playtime_forever': 'sum'}).reset_index()
    df['playtime_forever'] = round(df['playtime_forever']/60, 2)

    df_1 = df.groupby('user_id').agg({'playtime_forever' : 'sum'}).reset_index()
    ind = df_1['playtime_forever'].idxmax()
    usuario = df_1.loc[ind, 'user_id']
    df = df[df['user_id'] == usuario]
    df.rename(columns= {'playtime_forever': 'Horas jugadas'}, inplace=True)

    df.reset_index(inplace=True, drop=True)
    
    resultado = {
        f'Usuario con mas horas jugadas para el género {genero.replace("genre_", "")}:': df.loc[0,'user_id'],
        'Horas jugadas:': [{'Año:': df.loc[i,'Año'], 'Horas:': df.loc[i,'Horas jugadas'] } for i in range(len(df))]
    }

    return resultado
    

In [219]:
UserForGenre('ACTION')

{'Usuario con mas horas jugadas para el género action:': 'DownSyndromeKid',
 'Horas jugadas:': [{'Año: ': 1988.0, 'Horas: ': 11.07},
  {'Año: ': 1990.0, 'Horas: ': 0.0},
  {'Año: ': 1992.0, 'Horas: ': 0.0},
  {'Año: ': 1993.0, 'Horas: ': 0.0},
  {'Año: ': 1994.0, 'Horas: ': 0.0},
  {'Año: ': 1995.0, 'Horas: ': 8.72},
  {'Año: ': 1998.0, 'Horas: ': 0.0},
  {'Año: ': 2000.0, 'Horas: ': 1282.12},
  {'Año: ': 2001.0, 'Horas: ': 1.58},
  {'Año: ': 2002.0, 'Horas: ': 1230.48},
  {'Año: ': 2003.0, 'Horas: ': 1235.6},
  {'Año: ': 2004.0, 'Horas: ': 0.0},
  {'Año: ': 2005.0, 'Horas: ': 0.0},
  {'Año: ': 2006.0, 'Horas: ': 0.0},
  {'Año: ': 2007.0, 'Horas: ': 1231.67},
  {'Año: ': 2008.0, 'Horas: ': 1.58},
  {'Año: ': 2009.0, 'Horas: ': 0.0},
  {'Año: ': 2010.0, 'Horas: ': 1239.57},
  {'Año: ': 2011.0, 'Horas: ': 3716.58},
  {'Año: ': 2012.0, 'Horas: ': 4.42},
  {'Año: ': 2013.0, 'Horas: ': 61.17},
  {'Año: ': 2014.0, 'Horas: ': 234.22},
  {'Año: ': 2015.0, 'Horas: ': 406.08},
  {'Año: ': 2016.0

In [222]:
UserForGenre('asddee')

Género no encontrado. Ingrese un género válido.
Los géneros disponibles son:
["1990's", '2d', '3d platformer', 'accounting', 'action', 'action rpg', 'adventure', 'animation &amp; modeling', 'anime', 'arcade', 'atmospheric', 'audio production', 'based on a novel', "beat 'em up", 'bullet hell', 'casual', 'character customization', 'chess', 'choose your own adventure', 'classic', 'co-op', 'colorful', 'comedy', 'comic book', 'controller', 'cyberpunk', 'dark', 'dark comedy', 'dark fantasy', 'dark humor', 'design & illustration', 'design &amp; illustration', 'destruction', 'detective', 'difficult', 'documentary', 'early access', 'education', 'episodic', 'exploration', 'fps', 'family friendly', 'fantasy', 'fast-paced', 'female protagonist', 'first-person', 'free to play', 'funny', 'futuristic', 'game development', 'gore', 'great soundtrack', 'hack and slash', 'hidden object', 'historical', 'horror', 'indie', 'isometric', 'jrpg', 'local co-op', 'local multiplayer', 'lovecraftian', 'magic', 'ma

In [223]:
UserForGenre('Racing')

{'Usuario con mas horas jugadas para el género racing:': '76561198030382508',
 'Horas jugadas:': [{'Año:': 2010.0, 'Horas:': 3859.92}]}

def best_developer_year( año : int ):   
Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado.   
(reviews.recommend = True y comentarios positivos)  

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]  


In [185]:
def best_developer_year(año: int):

    try:
        año = int(año)
    except Exception as e:
        return print(f'Error {e}. Debe insertar un número entero.')
    
    df_sent = pd.read_parquet('../sentiment_analysis_2.parquet', columns= ['item_id', 'sentiment_analysis_2', 'recommend', 'Año'])

    años = list(df_sent['Año'].unique())

    if año not in años:
        print(f'No hay registros del año {año}')
        print(f'Los años disponibles son: {años}')
        return None
    
    df_steam = pd.read_parquet('../CleanData/steam_games.parquet', columns= ['id', 'developer'])

    df = df_sent.merge(df_steam, how='left', left_on='item_id', right_on='id')
    df.drop(columns=['item_id', 'id'], inplace=True)
    df.rename(columns= {'sentiment_analysis_2': 'rating'}, inplace=True)

    df = df.groupby(['developer', 'Año']).agg({'rating': (lambda x: (x == 2).sum()), 'recommend': 'sum'}).reset_index()
    df = df[df['Año'] == año]
    
    df['puntaje'] = df['rating'] + df['recommend']

    df.sort_values(by= 'puntaje', ascending=False, inplace=True)
    df.reset_index(inplace=True, drop= True)
    
    result = {
                'Puesto 1': df.loc[0, 'developer'],
                'Puesto 2': df.loc[1, 'developer'],
                'Puesto 3': df.loc[2, 'developer']
            }

    return result

In [177]:
best_developer_year(2011)

{'Puesto 1': 'Valve',
 'Puesto 2': 'Re-Logic',
 'Puesto 3': 'Tripwire Interactive'}

In [186]:
best_developer_year('dos mil once')

Error invalid literal for int() with base 10: 'dos mil once'. Debe insertar un número entero.


def developer_reviews_analysis( desarrolladora : str ):   
Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.  
Ejemplo de retorno:   
{'Valve' : [Negative = 182, Positive = 278]}

In [200]:
def developer_reviews_analysis(desarrolladora: str):

    if not isinstance(desarrolladora, str):
        return print('Debe ingresar una cadena de texto')
    
    desarrolladora = desarrolladora.lower()
    
    df_steam = pd.read_parquet('../CleanData/steam_games.parquet', columns= ['id', 'developer'])
    df_steam['developer'] = df_steam['developer'].apply(lambda x: x.lower())

    developers = list(df_steam['developer'].unique())
    
    if desarrolladora not in developers:
        print('Desarrolladora no encontrada. Por favor ingrese una desarrolladora válida.')
        print('Desarrolladoras disponibles:')
        print(developers)
        return None
    
    df_sent = pd.read_parquet('../sentiment_analysis_2.parquet', columns= ['item_id', 'sentiment_analysis_2'])

    df = df_sent.merge(df_steam,how= 'left', left_on= 'item_id', right_on='id')
    df.drop(columns=['id', 'item_id'], inplace=True)

    resultado = df[df['developer'] == desarrolladora]['sentiment_analysis_2'].value_counts()

    resultado_dic = {
        desarrolladora: [f'Negative = {resultado[0]}', f'Positive = {resultado[2]}']
    }

    return resultado_dic

In [198]:
developer_reviews_analysis('Tripwire Interactive')

{'Tripwire Interactive': ['Negative = 115', 'Positive = 318']}

In [199]:
developer_reviews_analysis('Valve')

{'Valve': ['Negative = 1095', 'Positive = 6274']}

In [201]:
developer_reviews_analysis('una desarrolladora inventada')

Desarrolladora no encontrada. Por favor ingrese una desarrolladora válida.
Desarrolladoras disponibles:
['kotoshiro', 'secret level srl', 'poolians.com', '彼岸领域', 'trickjump games ltd', 'poppermost productions', 'rewindapp', 'qucheza', 'stegalosaurus game development', 'copperpick studio', 'ghulam jewel', 'apillo', 'tero lunkka', 'frozenpepper', 'casey labrack', 'stainless games ltd', 'valve', 'dev4play', 'blueline games', 'games by brundle', 'etggames', 'projectorgames', 'unknown indie devs', 'the commander', 'lalalazero,urbanoff', 'strategy first', 'outerlight ltd.', 'zachary m thomas', 'breakout studio', 'zha7,phaedruns,yingke wu', 'ultraint', 'seth albertus', 'tpm.co soft works', 'wonderbox games', 'mad unicorn games', 'emanon studios', 'glyphx games', 'introversion software', 'facepunch studios', 'simbin', 'unknown worlds entertainment', 'cinemax, s.r.o.', 'sick puppies', 'deepred', 'funcom', 'bugbear entertainment', 'capcom co., ltd.', 'firefly studios,firaxis games', 'arkane stud

In [202]:
developer_reviews_analysis(115)

Debe ingresar una cadena de texto
